In [1]:
import os
from functools import partial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import AutoETS
from statsmodels.tsa.seasonal import seasonal_decompose

from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mae, mape, mase
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Markdown, Image

from fpppy.utils import plot_series

# endregion 
from plotnine import *

# change working directory 
# Check if current directory has any .ipynb files
if any(fname.endswith(".ipynb") for fname in os.listdir()):
    os.chdir("..")

# load local package modules

%load_ext autoreload
%autoreload 2

## 1.3. Some simpler time series models

Technically, a **time series model** provides a complete probabilistic model  that specifies all the joint distributions of random vectors ($X_1, ..., X_n$). That is, the probability at each *n* time point that the variable ($X_n$) would be less than or equal to any possible value of $- \infty < x_1, ..., x_n < \infty$. Mathematically, 

$$
P[X_1 \le x_1,..., X_n \le x_n], \infty < x_1, ..., x_n < \infty. n=, 1,2,...
$$

Because this is not practically feasible, only the mean and variance of the joint distributions are modelled (i..e, second-order properties). The theory of minimium mean sqare errors justifies use of second-order properties and guarantees that little information is lost. 

### 1.3.2. Models with trend and seasonality 



In [34]:
df_us_pop

,y,unique_id,ds,population_million,pop_squared
0,3929214,us_pop,1790-01-01,3.929214,15.438723
1,5308483,us_pop,1800-01-01,5.308483,28.179992
2,7239881,us_pop,1810-01-01,7.239881,52.415877
3,9638453,us_pop,1820-01-01,9.638453,92.899776
4,12860702,us_pop,1830-01-01,12.860702,165.397656
5,17063353,us_pop,1840-01-01,17.063353,291.158016
6,23191876,us_pop,1850-01-01,23.191876,537.863112
7,31443321,us_pop,1860-01-01,31.443321,988.682436
8,38558371,us_pop,1870-01-01,38.558371,1486.747974
9,50189209,us_pop,1880-01-01,50.189209,2518.956700


In [43]:
# Example 1.3.4. Population of the US with time-varying trend component 
df_us_pop = pd.read_csv('data/raw/uspop.tsm', names=['y'])

# add standard columns needed by nixtla functions 
df_us_pop['unique_id'] = 'us_pop'
n_obs = len(df_us_pop)
df_us_pop['ds'] = pd.date_range(start='1790-01-01', periods=n_obs, freq='10YS')

# add pop
df_us_pop['y'] = df_us_pop['y']/1e6
df_us_pop['pop_squared'] = df_us_pop['y']**2

from fpppy.models import LinearRegression
from mlforecast import MLForecast
from scipy import stats

mf = MLForecast(models=LinearRegression(), freq='10Y')

# Fit model
mf.fit(df_us_pop, fitted=True, static_features=[])

print_regression_summary_from_model(mf.models_['LinearRegression'])


MLForecast(models=[LinearRegression], freq=10Y, lag_features=[], date_features=[], num_threads=1)

#> Residuals:
#>     Min      1Q  Median      3Q     Max 
#> -35.2445 -19.6587  0.6536 20.6940 29.4927

#> Coefficients:
#>                Estimate  Std. Error   t value  Pr(>|t|)
#>   (Intercept)   31.8453      6.1181      5.21  5.04e-05 ***
#>   pop_squared    0.0041      0.0003     14.91  6.11e-12 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 22.611 on 19 degrees of freedom
Multiple R-squared: 0.921,   Adjusted R-squared: 0.917
F-statistic: 36367861539708.1 on 6.113332062795962e-12 and 19 DF, p-value: 1.11e-16


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names


In [39]:
def print_regression_summary_from_model(model):
    X = model._X.values.astype(float)
    y = model._y
    residuals = model._residuals

    n, p = X.shape
    X_design = np.hstack([np.ones((n, 1)), X])
    df = n - p - 1

    res_summary = np.percentile(residuals, [0, 25, 50, 75, 100])
    print("#> Residuals:")
    print(f"#>     Min      1Q  Median      3Q     Max ")
    print(f"#> {res_summary[0]:7.4f} {res_summary[1]:7.4f} "
          f"{res_summary[2]:7.4f} {res_summary[3]:7.4f} "
          f"{res_summary[4]:7.4f}\n")

    coef = np.insert(model.coef_, 0, model.intercept_)
    rss = np.sum(residuals ** 2)
    mse = rss / df
    var_beta = mse * np.linalg.inv(X_design.T @ X_design).diagonal()
    se_beta = np.sqrt(var_beta)
    t_stats = coef / se_beta
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df))

    print("#> Coefficients:")
    print(f"#> {'':>13} {'Estimate':>9} {'Std. Error':>11} {'t value':>9} "
          f"{'Pr(>|t|)':>9}")
    names = ['(Intercept)'] + model._var_names
    for name, est, se, t, p in zip(names, coef, se_beta, t_stats, p_values):
        stars = (
            '***' if p < 0.001 else
            '**' if p < 0.01 else
            '*' if p < 0.05 else
            '.' if p < 0.1 else
            ''
        )
        print(f"#> {name:>13} {est:9.4f} {se:11.4f} {t:9.2f} {p:9.3g} "
              f"{stars}")
    print("---")
    print("Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1")

    r_squared = model.score(X, y)
    adj_r_squared = 1 - (1 - r_squared) * (n - 1) / df
    f_stat = (r_squared / p) / ((1 - r_squared) / df)
    f_pval = 1 - stats.f.cdf(f_stat, p, df)

    print(f"\nResidual standard error: {np.sqrt(mse):.3f} "
          f"on {df} degrees of freedom")
    print(f"Multiple R-squared: {r_squared:.3f},   "
          f"Adjusted R-squared: {adj_r_squared:.3f}")
    print(f"F-statistic: {f_stat:.1f} on {p} and {df} DF, "
          f"p-value: {f_pval:.3g}")

print_regression_summary_from_model(mf.models_['LinearRegression'])

#> Residuals:
#>     Min      1Q  Median      3Q     Max 
#> -5.9347 -0.4801 -0.2052  1.3128  5.0941



NameError: name 'stats' is not defined